In [1]:
import pandas as pd
import numpy as np
import random
from rdkit import Chem
from snn_model import get_loss_fn
import torch
from torch.utils.data import TensorDataset, DataLoader
from snntorch import spikegen, surrogate
import matplotlib.pyplot as plt
from utils import load_dataset_df, fp_generator, smile_to_fp, data_splitter, get_spiking_net
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\snn_venv\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometri

In [2]:
#!pip install pandas numpy rdkit torch snntorch matplotlib scikit-learn deepchem pubchempy

In [3]:
#https://github.com/chainer/chainer-chemistry/blob/master/chainer_chemistry/dataset/splitters/scaffold_splitter.py

#splitter = ScaffoldSplitter()
#tasks, datasets, transformers = dc.molnet.load_bbbp(splitter=splitter, featurizer="ECFP", reload=True)
#train, val, test = datasets

#### Load DataFrame

In [4]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[1]
dirname = dt_file.removesuffix('.csv')

df, targets = load_dataset_df(filename=dt_file)

for t in targets:
    df_temp = df[[t, 'smiles']].dropna()
    class_counts = df[t].count()
    class_sum = df[t].sum()
    print(t, class_counts, round(class_sum/class_counts, 2)) 


Hepatobiliary disorders 1427 0.52
Metabolism and nutrition disorders 1427 0.7
Product issues 1427 0.02
Eye disorders 1427 0.61
Investigations 1427 0.81
Musculoskeletal and connective tissue disorders 1427 0.7
Gastrointestinal disorders 1427 0.91
Social circumstances 1427 0.18
Immune system disorders 1427 0.72
Reproductive system and breast disorders 1427 0.51
Neoplasms benign, malignant and unspecified (incl cysts and polyps) 1427 0.26
General disorders and administration site conditions 1427 0.91
Endocrine disorders 1427 0.23
Surgical and medical procedures 1427 0.15
Vascular disorders 1427 0.78
Blood and lymphatic system disorders 1427 0.62
Skin and subcutaneous tissue disorders 1427 0.92
Congenital, familial and genetic disorders 1427 0.18
Infections and infestations 1427 0.7
Respiratory, thoracic and mediastinal disorders 1427 0.74
Psychiatric disorders 1427 0.71
Renal and urinary disorders 1427 0.64
Pregnancy, puerperium and perinatal conditions 1427 0.09
Ear and labyrinth disorde

In [5]:
if dirname == 'tox21':
    # SR-ARE
    target_name = targets[0]
elif dirname == 'sider':
    #Hepatobiliary disorders 1427 samples, 0.52 class ratio
    target_name = targets[0]
else:
    target_name = targets[0]
    
df = df[[target_name, 'smiles']].dropna()

#### SMILE to Fingerprint

In [6]:
fp_types = [['morgan', 1024], ['maccs', 167], ['RDKit', 1024], ['pubchem', 881]]
fp_type, num_bits = fp_types[1]
#num_bits = 512
fp_config = {"fp_type": fp_type,
             "num_bits": num_bits}
#num_bits = 237
print(fp_type, '-', num_bits)

maccs - 167


In [7]:
dtype = torch.float32
split = "scaffold"
dataset = None
if dirname != 'BBBP':
    split = "random"
    fp_array, target_array = smile_to_fp(df, fp_config=fp_config, target_name=target_name)
    # Create Torch Dataset
    fp_tensor = torch.tensor(fp_array, dtype=dtype)
    target_tensor = torch.tensor(target_array, dtype=dtype).long()

    dataset = TensorDataset(fp_tensor, target_tensor)

[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:02] WARNING: not removing hydrogen atom without neighbors
[23:53:04] WARNING: not removing hydrogen atom without neighbors
[23:53:04] WARNING: not removing hydrogen atom without neighbors
[23:53:04] WARNING: not removing hydrogen atom without neighbors
[23:53:05] WARNING: not removing hydrogen atom without neighbors
[23:53:05] WARNING: not removing hydrogen atom without neighbors


#### Loss Function

In [8]:
from sklearn.utils.class_weight import compute_class_weight

loss_types = ['cross_entropy', 'rate_loss', 'temporal_loss']
loss_type = loss_types[0]
print(loss_type)


cross_entropy


#### Train Loop

In [9]:
net_types = ["SNN", "DSNN", "CSNN"]
net_type = net_types[0]
#spike_grad = surrogate.sigmoid(slope=25)
spike_grad = None
net_config = {"input_size": num_bits,
              "num_hidden": 512,
              "time_steps": 10,
              "spike_grad": spike_grad
              }

fast sigmoid / sigmoid
lr = 1e-5 ou -6, com 500-1000 epochs
batch size 16
batches devem incluir pos e neg samples
Adamax

experimentar com Tox21 NR-AR
temporal coding se não aumentar com rate coding

ver oversampling 

In [10]:
lr=1e-5 #default was e5-5
iterations = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 32
train_config = {"num_epochs": 500,
                "batch_size": batch_size,
                "device": device,
                "loss_type": loss_type,
                "loss_fn": None,
                'dtype': dtype,
                'num_steps': net_config['time_steps'],
                'val_net': None
                }
results = [[], [], [], [], [], []]

cuda


In [11]:
from rdkit import RDLogger

# Disable RDKit logging
RDLogger.DisableLog('rdApp.*')


In [12]:
for iter in range(iterations):
    print(f"Iteration:{iter + 1}/{iterations}")
    random.seed(iter+1)

    net, train_net, val_net, test_net = get_spiking_net(net_type, net_config)
    net = net.to(device)
    train_config['val_net'] = val_net
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=0)
    #experimentar weight decay = 1e-4 e subir (pouco, 1e-3)

    # DATA SPLIT
    train, val, test = data_splitter(df, target_name, split=split, dataset=dataset, fp_config=fp_config, seed=iter+1, dtype=dtype)
    _, train_label = train[:]
    _, val_label = val[:]
    _, test_label = test[:]
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

    # LOSS FN
    class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.array(train_label))
    #class_weights[0] = class_weights[0]/2 
    #class_weights[0] = class_weights[0]*2
    class_weights = torch.tensor(class_weights, dtype=torch.float)
    #class_weights = None
    train_config["loss_fn"] = get_loss_fn(loss_type=loss_type, class_weights=class_weights)

    # TRAINING
    net, loss_hist, val_acc_hist, val_auc_hist = train_net(net=net, optimizer=optimizer, train_loader=train_loader, val_loader=val_loader, train_config=train_config)

    # TESTING
    all_preds, all_targets = test_net(net, device, test_loader)
    print(len(test_loader))
    # METRICS
    accuracy = accuracy_score(all_targets, all_preds)
    auc_roc = roc_auc_score(all_targets, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_targets, all_preds).ravel()
    sensitivity = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = f1_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds)

    results[0].append(accuracy)
    results[1].append(auc_roc)
    results[2].append(sensitivity)
    results[3].append(specificity)
    results[4].append(f1)
    results[5].append(precision)


Iteration:1/1


C:\Users\knsve\AppData\Local\Temp\ipykernel_5988\1799220900.py:21: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.array(train_label))


Epoch:1


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument weight in method wrapper_CUDA_nll_loss_forward)

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.array([0, 1]), y=np.array(train_label))
class_weights[0] = class_weights[0]/2 
class_weights[1] = class_weights[1]*2
print(class_weights)

#### Smoothed Loss

In [ ]:
#from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
#from statsmodels.nonparametric.smoothers_lowess import lowess

print(loss_hist[len(loss_hist) - 5:len(loss_hist)])

fig = plt.figure(facecolor="w", figsize=(10, 5))
#plt.plot(np.convolve(loss_hist, np.ones(30)/30, mode='valid'))
#plt.plot(savgol_filter(loss_hist, window_length=100, polyorder=3))
#plt.plot(lowess(loss_hist, np.arange(len(loss_hist)), frac=0.1)[:, 1])
plt.plot(gaussian_filter1d(loss_hist, sigma=6))
#plt.plot(loss_hist)
plt.axhline(y=1, color='r', linestyle='--', label='y = 1')
plt.title("Loss Curve")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [14]:
# Validation Set
fig = plt.figure(facecolor="w", figsize=(10, 5))

plt.plot(gaussian_filter1d(val_auc_hist, sigma=6))
#plt.plot(val_auc_hist)
plt.axhline(y=1, color='r', linestyle='--', label='y = 1')
plt.title("ROC AUC every 5 epochs.")
plt.xlabel("Iteration")
plt.ylabel("ROC-AUC")
plt.legend()
plt.show()

#### Save Metrics

In [ ]:
metrics_np = np.zeros(12)

for i, metric in enumerate(results):
    metrics_np[i*2] = np.round(np.mean(metric), 3)
    metrics_np[i*2+1] = np.round(np.std(metric), 3)

# Print Results
print(f"Accuracy:  {metrics_np[0]:.3f} ± {metrics_np[1]:.3f}")
print(f"AUC ROC: {metrics_np[2]:.3f} ± {metrics_np[3]:.3f}")
print(f"Sensitivity: {metrics_np[4]:.3f} ± {metrics_np[5]:.3f}")
print(f"Specificity: {metrics_np[6]:.3f} ± {metrics_np[7]:.3f}")

metric_names = ['Acc', 'AUC', 'Sn', 'Sp', 'F1', 'Precision']
metrics_np = metrics_np.reshape(1, -1)
columns = []
for name in metric_names:
    columns.extend([f'Mean {name}', f'Std {name}'])


df_metrics = pd.DataFrame(metrics_np, columns=columns)
num_hidden = net_config['num_hidden']
time_steps = train_config['num_steps']
num_epochs = train_config['num_epochs']

if dirname == 'BBBP':
    if net_type == 'CSNN':
        filename = f"results\\{dirname}\\{net_type}_{fp_type}_{num_bits}_{time_steps}_lr{lr}_e{num_epochs}_{loss_type}_{split}.csv"
    else:
        filename = f"results\\{dirname}\\{net_type}_{fp_type}_{num_bits}_{num_hidden}_{time_steps}_lr{lr}_e{num_epochs}_{loss_type}_{split}.csv"
else:
    filename = f"results\\{dirname}\\{target_name}_{net_type}_{fp_type}_{num_bits}_{num_hidden}_{time_steps}_lr{lr}_e{num_epochs}_{loss_type}_{split}.csv"


df_metrics.to_csv(filename, index=False)

print(filename)

In [16]:
#from torchsummary import summary
#summary(net, input_size=(1, 1024),  batch_size=32)

#from torchinfo import summary

#summary(net, input_size=(batch_size, 1, 1024), verbose=0)